# Linear Algebra and Optimization for Machine Learning - Project

In [128]:
# imports
import numpy as np
import pandas as pd
import cvxpy as cp

### 1.

(a) Generate a $300 \times 20$ data matrix $X$, where each entry is uniformly random. Generate an outcome vector $y$, which is a linear combination of the columns of $X$ with uniformly random weights, and some Gaussian noise added to each entry of $y$.

In [129]:
# generate Xij ~ Unif([0, 1]), X.shape = (300, 20)

# shape
m = 300
n = 20

X = np.random.uniform(size = (m, n))

# generate random noise eps, eps.shape = (300,)
eps = np.random.normal(size = m)

# uniformly random weights weight, weight.shape = (20,)
weight = np.random.uniform(size = n)

# y = X weight + eps, y.shape = (300,)
y = X @ weight + eps

(b) Write a function to divide the data set into a train and test set.

In [194]:
# function for splitting into training and testing datasets
def train_test_split(X, y, p: float):
    '''
    (X, y): datapoints
    p: fraction of data that is in training set

    currently not random split --> implement later?

    returns: X_train, X_test, y_train, y_test
    '''

    m = X.shape[0]
    n = X.shape[1]

    split = int(np.ceil(p * m))

    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    return X_train, X_test, y_train, y_test

In [195]:
# example
p = 0.7
X_train, X_test, y_train, y_test = train_test_split(X, y, p)

(c) Write functions for OLS and Ridge regression and apply this to your synthetic data set. Discuss the performance on train and test sets.

In [182]:
def performance(X, y, w):
    MSE = np.mean((X @ w - y)**2)
    return MSE

# unsure if need to use other method to solve equation
def OLS_cvxpy(X_train, y_train):
    n = X_train.shape[1]
    
    w = cp.Variable(n)
    obj = cp.Minimize(cp.sum((X_train @ w - y_train)**2))

    prob = cp.Problem(obj)
    prob.solve()

    return w.value

def ridge_cvxpy(X_train, y_train, lam):
    n = X_train.shape[1]
    
    w = cp.Variable(n)
    obj = cp.Minimize(cp.sum((X_train @ w - y_train)**2) + lam * cp.sum(w**2))

    prob = cp.Problem(obj)
    prob.solve()

    return w.value

# def find_opt_lam()

In [183]:
# example usage
w_OLS = OLS_cvxpy(X_train, y_train)
w_ridge = ridge_cvxpy(X_train, y_train, 0.1)

# performance
perf_OLS_tr, perf_OLS_te = performance(X_train, y_train, w_OLS), performance(X_test, y_test, w_OLS)
perf_ridge_tr, perf_ridge_te = performance(X_train, y_train, w_ridge), performance(X_test, y_test, w_ridge)

print(f"OLS train MSE: {np.round(perf_OLS_tr, 4)}")
print(f"OLS test MSE: {np.round(perf_OLS_te, 4)}")
print("-" * 35)
print(f"ridge train MSE: {np.round(perf_ridge_tr, 4)}")
print(f"ridge test MSE: {np.round(perf_ridge_te, 4)}")

OLS train MSE: 0.8441
OLS test MSE: 1.1471
-----------------------------------
ridge train MSE: 0.8442
ridge test MSE: 1.1451


(d) Create a data matrix with many multicolinearities by adding a large number (say, 200) columns to X that are linear combinations of the original 20 columns with some Gaussian noise added to each entry. Run OLS and Ridge regression and discuss the performance on train and test sets. Is it hard to find a good value for $\lambda$?

In [184]:
def add_multicolinearity(X, num):
    n_feats = X.shape[1]
    n_entries = X.shape[0]

    new_cols = []
    
    for i in range(num):
        # uniformly random weights w, w.shape = (n_feats, )
        w = np.random.uniform(size = n_feats)

        # generate random noise eps, eps.shape = (n_entries,)
        eps = np.random.normal(size = n_entries)

        # linear combination of features + noise
        X_col = X @ w + eps
        new_cols.append(X_col.reshape(-1, 1))

    # concatenate the X with new columns
    X_new = np.hstack([X] + new_cols)
    return X_new

In [185]:
X_new = add_multicolinearity(X, 200)

X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y, p)

# example usage
w_OLS = OLS_cvxpy(X_train_new, y_train_new)
w_ridge = ridge_cvxpy(X_train_new, y_train_new, 0.1)

# performance
perf_OLS_tr, perf_OLS_te = performance(X_train_new, y_train_new, w_OLS), performance(X_test_new, y_test_new, w_OLS)
perf_ridge_tr, perf_ridge_te = performance(X_train_new, y_train_new, w_ridge), performance(X_test_new, y_test_new, w_ridge)

print(f"OLS train MSE: {np.round(perf_OLS_tr, 4)}")
print(f"OLS test MSE: {np.round(perf_OLS_te, 4)}")
print("-" * 35)
print(f"ridge train MSE: {np.round(perf_ridge_tr, 4)}")
print(f"ridge test MSE: {np.round(perf_ridge_te, 4)}")

# still need to find optimal lam

OLS train MSE: 0.0
OLS test MSE: 13.164
-----------------------------------
ridge train MSE: 0.0023
ridge test MSE: 11.6859


(e) Now instead of adding multicolinearities, add many superficial feature columns to X which have no relation to the outcome vector y. Again run OLS and Ridge regression and discuss the performance on train and test sets.

In [186]:
def add_superficial(X, num):
    n_entries = X.shape[0]

    sup_feats = np.random.uniform(size = (n_entries, num))
    print(sup_feats.shape)

    # concatenate the X with new columns
    X_new = np.hstack([X, sup_feats])
    return X_new

In [187]:
X_new = add_superficial(X, 200)

X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y, p)

# example usage
w_OLS = OLS_cvxpy(X_train_new, y_train_new)
w_ridge = ridge_cvxpy(X_train_new, y_train_new, 0.1)

# performance
perf_OLS_tr, perf_OLS_te = performance(X_train_new, y_train_new, w_OLS), performance(X_test_new, y_test_new, w_OLS)
perf_ridge_tr, perf_ridge_te = performance(X_train_new, y_train_new, w_ridge), performance(X_test_new, y_test_new, w_ridge)

print(f"OLS train MSE: {np.round(perf_OLS_tr, 4)}")
print(f"OLS test MSE: {np.round(perf_OLS_te, 4)}")
print("-" * 35)
print(f"ridge train MSE: {np.round(perf_ridge_tr, 4)}")
print(f"ridge test MSE: {np.round(perf_ridge_te, 4)}")

# still need to find optimal lam

(300, 200)
OLS train MSE: 0.0
OLS test MSE: 21.2017
-----------------------------------
ridge train MSE: 0.0228
ridge test MSE: 6.3495


## 2.

(a) Implement functions for logistic regression and hinge-loss classification.

In [188]:
# write func for M for classification

# y should be in {-1, 1} so need new y

def logistic_regression(X, y, lam):
    n, d = X.shape
    w = cp.Variable(d)

    # logistic loss: sum(log(1 + exp(-y_i * (X_i w))))
    loss = cp.sum(cp.logistic(-cp.multiply(y, X @ w)))
    reg = (lam / 2) * cp.sum_squares(w)

    prob = cp.Problem(cp.Minimize(loss + reg))
    prob.solve()

    return w.value

def hinge_loss(X, y, lam):
    n, d = X.shape
    w = cp.Variable(d)

    # hinge loss: sum(max(0, 1 - y_i * (X_i w)))
    loss = cp.sum(cp.pos(1 - cp.multiply(y, X @ w)))
    reg = (lam / 2) * cp.sum_squares(w)

    prob = cp.Problem(cp.Minimize(loss + reg))
    prob.solve()

    return w.value

In [190]:
w_test = hinge_loss(X_train, y_train, 0.7)
w_log = logistic_regression(X_train, y_train, 0.7)

array([0.05009248, 0.0448073 , 0.05339462, 0.02219443, 0.05266317,
       0.03364129, 0.03038353, 0.03868172, 0.01609537, 0.02674105,
       0.06448653, 0.05117883, 0.03610319, 0.07760338, 0.0542708 ,
       0.03220692, 0.05173475, 0.05957131, 0.02107843, 0.0306043 ])

(b) Create a random data matrix $X$ and construct an output vector $y$ by generating and a random weight vector $w$ and setting $y_i = \text{sign}(x^T_i w)$, where $x^T_i$ is the $i$-th row of $X$. Use a test/train split and check the performance of OLS, Ridge regression, logistic regression and hinge-loss classification for binary classification. Do you see a large difference in performance between these methods?

(c) Now create a data set $(X, y)$ for binary classification (with $X \in \mathbb{R}^{n \times d}$ and $y \in \{−1, 1\}^n$) such that, given a test/train split, OLS and Ridge perform very badly but logistic regression and hinge-loss classification perform well. What kind of properties of your data set are responsible for this?